In [ ]:
!pip install sdv

: 

In [ ]:
import numpy as np
import pandas as pd
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
from datetime import datetime, timedelta


In [ ]:
# Sensor Specifications
PL, PH = 10.0, 115.0  # Pressure range (kPa)
OL, OH = 0.400, 4.650  # Output voltage range (V)
A, B = 0.008095, -0.000952  # Gain and offset
VDD = 5.0  # Supply voltage

In [ ]:
# Define vehicle states with expected ranges
vehicle_states = [
    ("Engine OFF", (95, 105), (4.5, 4.9)),
    ("Idle", (20, 35), (0.9, 1.5)),
    ("Acceleration", (40, 70), (2.0, 3.5)),
    ("Throttle Release", (10, 30), (0.5, 1.3)),
    ("Wide Open Throttle", (95, 101), (4.0, 4.8))
]

In [ ]:
# Generate initial real-like data
num_samples = 5000
pressure_values = np.random.uniform(PL, PH, num_samples) + np.random.normal(0, 2, num_samples)  # Adding noise to pressure
voltage_values = ((A * pressure_values + B) * VDD) + np.random.normal(0, 0.1, num_samples)  # Adding noise to voltage

In [ ]:
# Ensure some voltage values are exactly 0 or 5
num_special_cases = int(0.15 * num_samples)
special_indices = np.random.choice(num_samples, num_special_cases, replace=False)
voltage_values[special_indices[:num_special_cases // 3]] = 0.0
voltage_values[special_indices[num_special_cases // 3:num_special_cases * 2 // 3]] = 5.0

In [ ]:
# Assign vehicle states
def get_vehicle_state(pressure, voltage):
    min_distance = float("inf")
    closest_state = "Unknown"

    for state, p_range, v_range in vehicle_states:
        p_mid = sum(p_range) / 2
        v_mid = sum(v_range) / 2
        distance = abs(pressure - p_mid) + abs(voltage - v_mid)

        if distance < min_distance:
            min_distance = distance
            closest_state = state

    return closest_state

In [ ]:
vehicle_state_values = [get_vehicle_state(p, v) for p, v in zip(pressure_values, voltage_values)]


In [ ]:
# Assign fault types
def get_fault_type(pressure, voltage):
    if voltage == 0.0:
        return "Wiring Issue"
    elif voltage == 5.0:
        return "Power Supply Issue"
    elif pressure < PL or pressure > PH:
        return "Sensor Fault"
    return "Normal"

In [ ]:
fault_types = [get_fault_type(p, v) for p, v in zip(pressure_values, voltage_values)]


In [ ]:
# Balance fault conditions
fault_counts = {"Normal": 2000, "Sensor Fault": 2000, "Wiring Issue": 2000, "Power Supply Issue": 2000}
selected_indices = []

for fault, count in fault_counts.items():
    indices = [i for i, f in enumerate(fault_types) if f == fault]
    if len(indices) >= count:
        selected_indices.extend(np.random.choice(indices, count, replace=False))
    else:
        selected_indices.extend(indices)
        additional_indices = np.random.choice(indices, count - len(indices), replace=True)
        selected_indices.extend(additional_indices)

pressure_values = np.array(pressure_values)[selected_indices]
voltage_values = np.array(voltage_values)[selected_indices]
vehicle_state_values = np.array(vehicle_state_values)[selected_indices]
fault_types = np.array(fault_types)[selected_indices]


In [ ]:
# Reduce normal condition
normal_indices = [i for i, f in enumerate(fault_types) if f == "Normal"]
fault_indices = [i for i, f in enumerate(fault_types) if f != "Normal"]
reduce_normal = int(0.4 * len(normal_indices))
selected_normals = np.random.choice(normal_indices, len(normal_indices) - reduce_normal, replace=False)
selected_indices = np.concatenate((selected_normals, fault_indices))

pressure_values = np.array(pressure_values)[selected_indices]
voltage_values = np.array(voltage_values)[selected_indices]
vehicle_state_values = np.array(vehicle_state_values)[selected_indices]
fault_types = np.array(fault_types)[selected_indices]

In [ ]:
# Generate timestamps
start_time = datetime.now()
timestamps = [start_time + timedelta(seconds=i) for i in range(len(selected_indices))]

In [ ]:
# Create DataFrame
real_data = pd.DataFrame({
    "Timestamp": timestamps,
    "Pressure (kPa)": pressure_values,
    "Voltage (V)": voltage_values,
    "Vehicle State": vehicle_state_values,
    "Fault Type": fault_types
})

In [ ]:
# Define metadata
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(real_data)
metadata.update_column("Vehicle State", sdtype="categorical")
metadata.update_column("Fault Type", sdtype="categorical")


In [ ]:
# Train CTGAN model
ctgan = CTGANSynthesizer(metadata=metadata)
ctgan.fit(real_data)


In [ ]:
# Generate synthetic data
synthetic_data = ctgan.sample(10000)

In [ ]:
# Ensure only valid vehicle states exist in synthetic data
valid_states = {state[0] for state in vehicle_states}
synthetic_data = synthetic_data[synthetic_data["Vehicle State"].isin(valid_states)]

In [ ]:
# Ensure fault types include all expected categories
valid_faults = {"Normal", "Sensor Fault", "Wiring Issue", "Power Supply Issue"}
synthetic_data = synthetic_data[synthetic_data["Fault Type"].isin(valid_faults)]


In [ ]:
# Display sample dataset
print(synthetic_data.head())


In [ ]:
from google.colab import files

# Save synthetic data to CSV
synthetic_data.to_csv("synthetic_dataset.csv", index=False)

# Download the file
files.download("synthetic_dataset.csv")